# Crop Segmentation and Yield Count using Graph based methods

### 1. Image segmentation using SLIC and Graph Cut method

In [9]:
import cv2
import sys
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
import maxflow
from scipy.spatial import Delaunay

# Calculate the SLIC superpixels, their histograms and neighbors
def superpixels_histograms_neighbors(img):
    # SLIC
    segments = slic(img, n_segments=500, compactness=18.5)
    segments_ids = np.unique(segments)

    # centers
    centers = np.array([np.mean(np.nonzero(segments==i),axis=1) for i in segments_ids])

    # H-S histograms for all superpixels
    hsv = cv2.cvtColor(img.astype('float32'), cv2.COLOR_BGR2HSV)
    bins = [20, 20] # H = S = 20
    ranges = [0, 360, 0, 1] # H: [0, 360], S: [0, 1]
    colors_hists = np.float32([cv2.calcHist([hsv],[0, 1], np.uint8(segments==i), bins, ranges).flatten() for i in segments_ids])

    # neighbors via Delaunay tesselation
    tri = Delaunay(centers)
    return (centers,colors_hists,segments,tri.vertex_neighbor_vertices)

# Get superpixels IDs for FG and BG from marking
def find_superpixels_under_marking(marking, superpixels):
    fg_segments = np.unique(superpixels[marking[:,:,0]!=255])
    bg_segments = np.unique(superpixels[marking[:,:,2]!=255])
    return (fg_segments, bg_segments)

# Sum up the histograms for a given selection of superpixel IDs, normalize
def cumulative_histogram_for_superpixels(ids, histograms):
    h = np.sum(histograms[ids],axis=0)
    return h / h.sum()

# Get a bool mask of the pixels for a given selection of superpixel IDs
def pixels_for_segment_selection(superpixels_labels, selection):
    pixels_mask = np.where(np.isin(superpixels_labels, selection), True, False)
    return pixels_mask

# Get a normalized version of the given histograms (divide by sum)
def normalize_histograms(histograms):
    return np.float32([h / h.sum() for h in histograms])

# Perform graph cut using superpixels histograms
def do_graph_cut(fgbg_hists, fgbg_superpixels, norm_hists, neighbors):
    num_nodes = norm_hists.shape[0]
    # Create a graph of N nodes, and estimate of 5 edges per node
    g = maxflow.Graph[float](num_nodes, num_nodes * 5)
    # Add N nodes
    nodes = g.add_nodes(num_nodes)

    hist_comp_alg = cv2.HISTCMP_KL_DIV

    # Smoothness term: cost between neighbors
    indptr,indices = neighbors
    for i in range(len(indptr)-1):
        N = indices[indptr[i]:indptr[i+1]] # list of neighbor superpixels
        hi = norm_hists[i]                 # histogram for center
        for n in N:
            if (n < 0) or (n > num_nodes):
                continue
            # Create two edges (forwards and backwards) with capacities based on
            # histogram matching
            hn = norm_hists[n]             # histogram for neighbor
            g.add_edge(nodes[i], nodes[n], 20-cv2.compareHist(hi, hn, hist_comp_alg),
                                           20-cv2.compareHist(hn, hi, hist_comp_alg))

    # Match term: cost to FG/BG
    for i,h in enumerate(norm_hists):
        if i in fgbg_superpixels[0]:
            g.add_tedge(nodes[i], 0, 1000) # FG - set high cost to BG
        elif i in fgbg_superpixels[1]:
            g.add_tedge(nodes[i], 1000, 0) # BG - set high cost to FG
        else:
            g.add_tedge(nodes[i], cv2.compareHist(fgbg_hists[0], h, hist_comp_alg),cv2.compareHist(fgbg_hists[1], h, hist_comp_alg))

    g.maxflow()
    return g.get_grid_segments(nodes)

# Function to handle mouse movements
def draw_segment(event,x,y,flags,param):
    global refPt, haveBG, haveFG, newMask

    # Store the point when mouse dragging is started
    if event == cv2.EVENT_MBUTTONDOWN or event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x,y)]
    # Draw line between mouse start and end. Update segmentation if both foreground and background markings are present.
    if event == cv2.EVENT_MBUTTONUP or event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        if fgColor == True:    # Mode = foreground
            haveFG = True
            cv2.line(image,refPt[0],refPt[1],(0,0,255),5)    # Draw Red line
            cv2.line(newMask,refPt[0],refPt[1],(0,0,255),5)
        elif bgColor == True:    # Mode = background
            haveBG = True
            cv2.line(image,refPt[0],refPt[1],(255,0,0),5)    # Draw Blue line
            cv2.line(newMask,refPt[0],refPt[1],(255,0,0),5)
        # If both foreground and backgroudn markings are available then generate the binary image with foreground and background
        if haveBG == True and haveFG == True:
            #Calculating Foreground and Background superpixels using marking
            fg_segments, bg_segments = find_superpixels_under_marking(newMask, superpixels)
            
            #Calculating color histograms for FG
            fg_cumulative_hist = cumulative_histogram_for_superpixels(fg_segments, color_hists)
            
            #Calculating color histograms for BG
            bg_cumulative_hist = cumulative_histogram_for_superpixels(bg_segments, color_hists)
            
            #Construct a graph that takes into account superpixel-to-superpixel interaction (smoothness term), as well as superpixel-FG/BG interaction (match term)
            graph_cut = do_graph_cut([fg_cumulative_hist,bg_cumulative_hist], [fg_segments,bg_segments], norm_hists, neighbors)
            
            mask = pixels_for_segment_selection(superpixels, np.nonzero(graph_cut))
            # mask is bool, conver 1 to 255 and 0 will remain 0 for displaying purpose
            mask = np.uint8(mask * 255)
            output_name = Path(input_path).stem
            #thr, bin = cv2.threshold(mask, 120, 255.0, cv2.THRESH_BINARY);
            #print("threshold: ",thr)
            mask = cv2.resize(mask, (img.shape[1], img.shape[0]))
            result = cv2.bitwise_or(img, img, mask=mask);
            res_name = output_name+'_segment.png'
            path = '/home/captainvish/Project/Final Year/Output'
            cv2.imwrite(os.path.join(path,res_name), result)
            # Show binary image
            cv2.imshow(res_name,mask)
            pathmask = '/home/captainvish/Project/Final Year/Mask'
            mask_name = output_name+'_mask.png'
            cv2.imwrite(os.path.join(pathmask,mask_name), mask)

if __name__ == '__main__':
    
    list_dr = '/home/captainvish/Project/Final Year/Input'
    for image in os.listdir(list_dr):  
        # Reading image
        input_path = os.path.join(list_dr, image)
        img = cv2.imread(input_path, cv2.IMREAD_COLOR)
        imgname = image
        height, width,band = img.shape
        print("Height:  " + str(height))
        print("Width:   " + str(width))
        
        #Calculating SLIC over image
        centers, color_hists, superpixels, neighbors = superpixels_histograms_neighbors(img)
        norm_hists = normalize_histograms(color_hists)

        image = img.copy()

        cv2.namedWindow(imgname)
        cv2.setMouseCallback(imgname,draw_segment)
        fgColor = True
        bgColor = False
        haveFG = False
        haveBG = False
        #Create a mask with only markings used in segmentation in original image.
        newMask = np.zeros((img.shape), np.uint8)
        # Fill image with white color(set each pixel to red).
        newMask[:] = (255, 255, 255)
        while(1):
            cv2.imshow(imgname,image)
            key = cv2.waitKey(1) & 0xFF
            # Stop is user press 'q'
            if key == ord("q"):
                break;
            if key == ord("s"):
                mask_name = Path(input_path).stem + '_mark.png'
                path = '/home/captainvish/Project/Final Year/Marking'
                cv2.imwrite(os.path.join(path,mask_name), newMask)
                break;
            # Switch to background mode if user press 'c'
            if key == ord("b"):
                bgColor = True
                fgColor = False
            # Switch to foreground mode if user press 'c'
            if key == ord("f"):
                fgColor = True
                bgColor = False
            # Reset markings if user press 'r'
            if key == ord("r"):
                image = img.copy()
                bgColor = False
                fgColor = True
                haveFG = False
                haveBG = False
                newMask[:] = (255, 255, 255)
                cv2.destroyWindow(imgname)
        cv2.destroyAllWindows()
        

    

Height:  720
Width:   720


/tmp/ipykernel_3991/3732865020.py:16: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  segments = slic(img, n_segments=500, compactness=18.5)


Height:  720
Width:   720
Height:  720
Width:   720


### 2. Yield Count using Watershed algorithm

In [10]:
# import the necessary packages
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import numpy as np
import argparse
import imutils
import cv2
import os

path = '/home/captainvish/Project/Final Year/Output'
for filename in os.listdir(path):
    input_path = os.path.join(path, filename)
    img = cv2.imread(input_path)
    shifted = cv2.pyrMeanShiftFiltering(img, 21, 51)

    # Otsu's thresholding
    gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    print(thresh)
    D = ndimage.distance_transform_edt(thresh)
    localMax = peak_local_max(D, indices=False, min_distance=27,labels=thresh)
    
    # perform a connected component analysis on the local peaks,
    # using 8-connectivity, then appy the Watershed algorithm
    markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
    labels = watershed(-D, markers, mask=thresh)
    print("{} Arecanuts found in the image segment.".format(len(np.unique(labels)) - 1))
    for label in np.unique(labels):
        # if the label is zero, we are examining the 'background'
        # so simply ignore it
        if label == 0:
            continue
        # otherwise, allocate memory for the label region and draw
        # it on the mask
        mask = np.zeros(gray.shape, dtype="uint8")
        mask[labels == label] = 255
        # detect contours in the mask and grab the largest one
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        # draw a circle enclosing the object
        ((x, y), r) = cv2.minEnclosingCircle(c)
        #cv2.circle(img, (int(x), int(y)), int(r), (0, 255, 0), 2)
        cv2.putText(img, "#{}".format(label), (int(x) - 10, int(y)),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    # show the output image
    yield_name = Path(input_path).stem + '_count.png'
    path1 = '/home/captainvish/Project/Final Year/Yield Count'
    cv2.imwrite(os.path.join(path1,yield_name), img)
    cv2.imshow(yield_name, img)
    cv2.waitKey(0)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
40 Arecanuts found in the image segment.


/tmp/ipykernel_3991/811683421.py:22: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  localMax = peak_local_max(D, indices=False, min_distance=27,labels=thresh)
/home/captainvish/.local/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [255   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
35 Arecanuts found in the image segment.


/tmp/ipykernel_3991/811683421.py:22: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  localMax = peak_local_max(D, indices=False, min_distance=27,labels=thresh)
/home/captainvish/.local/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


[[255 255 255 ...   0   0   0]
 [255 255 255 ...   0   0   0]
 [255 255 255 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
34 Arecanuts found in the image segment.


/tmp/ipykernel_3991/811683421.py:22: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  localMax = peak_local_max(D, indices=False, min_distance=27,labels=thresh)
/home/captainvish/.local/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
